In [55]:
import torch 
from torch.utils.data import Dataset, DataLoader 
import numpy as np 
import os 
# import pickle
import joblib

In [56]:
labels_col = 0 
raw_data_path = '/home/kuba/Documents/Data/Raw/sf_nyc_homes.csv'    #THIS NEEDS TO BE ABSOLUTE PATH'''
train_percent = 0.7
test_percent = 1 - train_percent
path_to_save_dir = '/home/kuba/Documents/Data/Datasets/SF_NYC_Homes'  #path to save the data too'''


In [57]:
class TrainDataSet():
    def __init__(self, raw_data_path, train_percent, labels_col):
        data = np.loadtxt(raw_data_path, delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = data.shape[0]

        n_train_samples = int(self.n_samples * train_percent)
        self.rows = np.random.permutation(self.n_samples)[:n_train_samples]

        self.x = torch.from_numpy(data[self.rows, :labels_col])
        self.y = torch.from_numpy(data[self.rows, [labels_col]])
        self.n_train_samples = n_train_samples

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_train_samples
    
class TestDataSet():
    def __init__(self, raw_data_path, test_percent, labels_col, train_rows):
        data = np.loadtxt(raw_data_path, delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = data.shape[0]

        n_train_samples = int(self.n_samples * test_percent)
        rows = ~train_rows

        self.x = torch.from_numpy(data[rows, :labels_col])
        self.y = torch.from_numpy(data[rows, [labels_col]])
        self.n_train_samples = n_train_samples

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_train_samples
  

In [58]:
def save_dataset(dataset, path_to_save_dir, file_name):
    os.makedirs(path_to_save_dir,exist_ok=True)
    with open(os.path.join(path_to_save_dir,file_name), 'wb') as f:
        joblib.dump(dataset, f)


In [59]:
#save to non raw data to data Datasets
train_set = TrainDataSet(raw_data_path, train_percent, labels_col)
test_set = TestDataSet(raw_data_path, train_percent, labels_col, train_set.rows)

save_dataset(train_set, path_to_save_dir, 'train.pkl')
save_dataset(test_set, path_to_save_dir, 'test.pkl')

In [60]:
#test